In [8]:
import kagglehub
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
from torchvision.datasets import ImageFolder

In [9]:
dataset_path = kagglehub.dataset_download('bhavikjikadara/dog-and-cat-classification-dataset')
directory = os.path.join(dataset_path, 'PetImages')

In [11]:
alexnet_transforms = transforms.Compose([
    transforms.Resize((224, 224)), # на фото такого размера обучалась
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [13]:
dataset = ImageFolder(root=directory, transform=alexnet_transforms)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

In [14]:
import torch.nn as nn

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [19]:
features = nn.Sequential(
    nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
    nn.ReLU(inplace=True),
    nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2), 
    nn.MaxPool2d(kernel_size=3, stride=2),

    nn.Conv2d(96, 256, kernel_size=5, padding=2),
    nn.ReLU(inplace=True),
    nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
    nn.MaxPool2d(kernel_size=3, stride=2),

    nn.Conv2d(256, 384, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),

    nn.Conv2d(384, 384, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),

    nn.Conv2d(384, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
)

In [20]:
classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(6*6*256, 4096),
    nn.ReLU(inplace=True),
    
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    
    nn.Linear(4096, 2),
)

In [21]:
alexnet_model = nn.Sequential(
    features,
    nn.Flatten(),
    classifier
).to(device)

In [22]:
import torch.optim as optim
import time

In [23]:
def check_accuracy(loader, model, device):
    
    model.eval() 
    
    num_correct = 0
    num_samples = 0
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            scores = model(x)
            
            _, predictions = scores.max(dim=1) 
            
            num_correct += (predictions == y).sum().item()
            num_samples += predictions.size(0)
            
    accuracy = (num_correct / num_samples) * 100
    
    model.train() 
    
    return accuracy

In [24]:
def train_epoch(model, loader, criterion, optimizer, device):
    
    model.train()
    running_loss = 0.0
    
    loop = tqdm(loader, desc='Training', leave=True)
    
    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device)
        targets = targets.to(device)
        
        scores = model(data)
        loss = criterion(scores, targets)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        
        loop.set_postfix(loss=loss.item())

    avg_loss = running_loss / len(loader)
    return avg_loss

In [26]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(alexnet_model.parameters(), lr=0.0001)

test_accuracy_before = check_accuracy(test_loader, alexnet_model, device)
print(f"pred train {test_accuracy_before:.2f}%")
print("\n")

pred train 50.38%




In [28]:
print("количество эпох: 10")

start_time = time.time()

for epoch in range(1,  11):
    print(f"\n--- epoch {epoch}/{10} ---")
    
    train_loss = train_epoch(
        alexnet_model, 
        train_loader, 
        criterion, 
        optimizer, 
        device
    )
    
    test_accuracy = check_accuracy(test_loader, alexnet_model, device)
    
    print(f"avg loss per epoch: {train_loss:.4f}")
    print(f"accuracy on test: {test_accuracy:.2f}%")

end_time = time.time()
total_time = end_time - start_time

print(f"total train time {total_time:.2f} sec")

количество эпох: 10

--- epoch 1/10 ---


Training: 100%|███████████████████████████████████████████████████████████| 313/313 [58:05<00:00, 11.14s/it, loss=0.63]


KeyboardInterrupt: 